In [9]:
import numpy as np
from functools import partial
import torch
from lib.timer import Timer
from lib.utils import load_obj, natural_key
from datasets.indoor import IndoorDataset
from datasets.modelnet import get_train_datasets, get_test_datasets
import os,re,sys,json,yaml,random, argparse, torch, pickle
from easydict import EasyDict as edict
from configs.models import architectures
from models.architectures import KPFCNN
import open3d as o3d
import matplotlib.pyplot as plt

def load_config(path):
    """
    Loads config file:

    Args:
        path (str): path to the config file

    Returns: 
        config (dict): dictionary of the configuration parameters, merge sub_dicts

    """
    with open(path,'r') as f:
        cfg = yaml.safe_load(f)
    
    config = dict()
    for key, value in cfg.items():
        for k,v in value.items():
            config[k] = v

    return config


def get_datasets(config):
    info_train = load_obj(config.train_info)
    train_set = IndoorDataset(info_train,config,data_augmentation=True)
    return train_set



def save_2d_ply(filename, points_2d, colors=None):
    with open(filename, 'w') as f:
        # 写入头部信息
        f.write("ply\n")
        f.write("format ascii 1.0\n")
        f.write(f"element vertex {len(points_2d)}\n")
        f.write("property float x\n")
        f.write("property float y\n")
        f.write("property float z\n")  # 添加一个 z 属性，值全为 0
        if colors is not None:
            f.write("property uchar red\n")
            f.write("property uchar green\n")
            f.write("property uchar blue\n")
        f.write("end_header\n")
        
        # 写入点云数据
        for i, point in enumerate(points_2d):
            line = f"{point[0]} {point[1]} 0.0"
            if colors is not None:
                color = colors[i]
                line += f" {int(color[0] * 255)} {int(color[1] * 255)} {int(color[2] * 255)}"
            f.write(line + "\n")

if __name__ == '__main__':
  

    # 生成随机点云数据
    point_cloud_np = np.random.rand(100, 3)

    # 将numpy数组转换为Open3D的PointCloud对象
    point_cloud_o3d = o3d.geometry.PointCloud()
    point_cloud_o3d.points = o3d.utility.Vector3dVector(point_cloud_np)
    pcd = o3d.io.read_point_cloud("./ply/point_cloud_src_pcd_o3d.ply")

    # 使用draw_plotly可视化点云
    o3d.visualization.draw_plotly([pcd], mesh_show_wireframe=False)

